In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
#from src.scripts.process_tweets import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
from collections import defaultdict
import seaborn as sns
%matplotlib inline

In [2]:
# Loop through cities
tweet_dir= Path("../data/processed/park_user_tweets")
cities = list(tweet_dir.glob("*.json"))

In [3]:
## load all stop wordds
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)

In [5]:
x = 5
print(cities[5].stem)
with open(cities[5], 'r') as f:
    park_user_tweets = json.load(f)

TX_El_Paso_4824000


In [7]:
## test time control 
park_tweet_text, control_tweet_text, park_list = get_time_control_text(park_user_tweets)
park_stops  = get_park_stopwords(park_list)
stop_list = park_stops+stop_dict['all_cities']+ stop_dict[cities[5].stem]
#bootstrap_sentiment(park_tweet_text, control_tweet_text, stop_list,sample=.8,runs=5)

## sensitivity tests and stop word check ##

In [100]:
# Now we bootstrap a vector of 100 seniments, by choosing 80% of the tweets in each group
for n in np.arange(.1,1,.1):
    bumps = bootstrap_sentiment(park_tweet_text, control_tweet_text, stop_list,sample=n,runs=5)
    print("{:.0f}% sampled: Mean Park: {:.4f} with range {:.4f}".format(n*100, np.mean(bumps), max(bumps)-min(bumps)))
#print(min(park_sentis), np.mean(park_sentis), max(park_sentis))
#print(min(control_sentis), np.mean(control_sentis), max(control_sentis))

10% sampled: Mean Park: 0.2728 with range 0.1067
20% sampled: Mean Park: 0.2534 with range 0.0409
30% sampled: Mean Park: 0.2628 with range 0.1100
40% sampled: Mean Park: 0.2694 with range 0.0546
50% sampled: Mean Park: 0.2564 with range 0.0277
60% sampled: Mean Park: 0.2592 with range 0.0227
70% sampled: Mean Park: 0.2654 with range 0.0103
80% sampled: Mean Park: 0.2642 with range 0.0283
90% sampled: Mean Park: 0.2682 with range 0.0193


In [137]:
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
stop_lower = [x.lower() for x in dict(regex.get_ngrams(' '.join(stop_list), path='../src/func/ngrams.bin')).keys()]
for word in stop_lower:
    if word in labMTwordList:
        i = labMTwordList.index(word)
        val = labMTvector[i]
        if val < 4 or val > 6:
            pass
            #print(word, labMTvector[i])